In [1]:
import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve

# 測試的

In [2]:
def f(x):
    return x + 1000
 
def g(y):
    return y * 3

def bestweight_pre(w):
    w0 = w[0]
    w1 = w[1]
    return [
        f(w0) + 3*g(w0) - 1,
        f(w1) + 3*g(w1) - 1
    ]

r = fsolve(bestweight_pre, [0,0])
print(r)

[-99.9 -99.9]


D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


# 開發中的

In [3]:
def f1(x,arr_returns):
    arr_returns - rfrate 
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)
def get_riskiness_r(arr_returns):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

In [4]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def finalrisk(weight,all_return):
    w = weight
    a = all_return
    portreturn = np.dot(np.array(all_return),np.array(weight).T)
    return risk(portreturn)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))

# 這是原本峻安要的輸出的內容，是以4個weight值為例
def bestweight_sample(w):
    return [
        risk(w[0]*all_return.iloc[:,0]) - np.mean(all_return.iloc[:,0])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,0]*w[0]) , 
        risk(w[1]*all_return.iloc[:,1]) - np.mean(all_return.iloc[:,1])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,1]*w[1]) , 
        risk(w[2]*all_return.iloc[:,2]) - np.mean(all_return.iloc[:,2])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,2]*w[2]) , 
        risk(w[3]*all_return.iloc[:,3]) - np.mean(all_return.iloc[:,3])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,3]*w[3])
    ]



# 比較好的寫法，沒有限制weight的數量(亦即ETF的支數)
def bestweight(w):
    r = []
    restrict = sum(w) - 1
    r.append(restrict)
    for i in range(len(w)-1):
        v =(risk(w[i]*all_return.iloc[:,i]) \
            - (np.mean(all_return.iloc[:,i])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i]*w[i])-rfrate) )**4 \
            + (risk(w[i+1]*all_return.iloc[:,i+1]) \
            - (np.mean(all_return.iloc[:,i-1])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i+1]*w[i+1])-rfrate) )**4
        r.append(v) 
    return r

In [5]:
d1 = [-0.1,0.2,0.3,0.4]
d2 = [0.1,-0.2,-0.3,0.4]
d3 = [-0.1,0.3,0.4,0.5]
d4 = [-0.2,-0.4,0.5,0.6]
d5 = [-0.2,-0.4,0.6,0.8]
# d1-d5共有五個資產

rfrate = 0.2 

# 這邊應該是要再多一個轉置，意即pd.DataFrame([d1,d2,d3,d4,d5]).T
all_return = pd.DataFrame([d1,d2,d3,d4,d5]).T
print(all_return.shape)

# weight應該有五位數
weight = [0.1,0.2,0.2,0.5,-0.1]

#r1 = fsolve(bestweight_sample, weight)
#print(r1)

r2 = fsolve(bestweight, weight)
print(r2)
all_return

D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


(4, 5)


D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


[ 0.1  0.2  0.2  0.5 -0.1]


,0,1,2,3,4
0,-0.1,0.1,-0.1,-0.2,-0.2
1,0.2,-0.2,0.3,-0.4,-0.4
2,0.3,-0.3,0.4,0.5,0.6
3,0.4,0.4,0.5,0.6,0.8


In [6]:
d1 = [0.049,0.05,0.05,0.06]
d2 = [0.028,0.03,0.07,0.04]
all_return = pd.DataFrame([d1,d2]).T
print(all_return.shape)


#weight = [0.1,0.2,0.2,0.5]
weight = [0.4,0.5]
all_return
r2 = fsolve(bestweight, weight)
print(r2)

(4, 2)


D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[0.18668277 0.60785989]


# 參考文章:
* [另外的python求解套件說明](https://github.com/wurmen/Gurobi-Python/blob/master/python-gurobi%20%20model/Python%2BGurobi%E5%BB%BA%E6%A8%A1.md)

# 比較sharpe ratio是否符合原本假設
## case1:正確解答

In [88]:
import numpy as np
x = np.array([-0.05,0.05,0.09,-0.01,0.04,0.01]*2)
y = np.array([0.1,-0.01,0.01,-0.02,0.02,-0.05]*2)
rfrate = 0.002

In [93]:
from scipy import optimize
def f(a):
    return np.std(a*x+(1-a)*y)/(np.sum(a*x+(1-a)*y)-rfrate) #倒數最小，原本值最大
abc = optimize.fmin(f, 0)
abc[0]

Optimization terminated successfully.
         Current function value: 0.132325
         Iterations: 23
         Function evaluations: 46


0.6063125000000006

## case2

<img src="picture/bestweight.PNG">


In [114]:
def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100
    else:
        return 0
def bestweight(w):
    r = []
    r.append(np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v)
    return r

In [112]:
def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v)
    return r

In [10]:
def risk(x):
    return np.std(x)

In [113]:
all_return = pd.DataFrame([x,y]).T
#print(all_return.shape)
weight = [0.1,0.1]

r2 = fsolve(bestweight, weight)
#for i in range(10):
#    r2 = fsolve(bestweight, r2)
print(r2)


權重: [0.1 0.1] 誤差值: 16.000000000000025
權重: [0.1 0.1] 誤差值: 16.000000000000025
權重: [0.1 0.1] 誤差值: 16.000000000000025
權重: [0.1 0.1] 誤差值: 15.999999233592906
權重: [0.1 0.1] 誤差值: 15.999999753128154
權重: [-0.23425188  1.23425187] 誤差值: 7.10741473189867
權重: [0.11591007 0.14724472] 誤差值: 8.706201727505066
權重: [0.15143398 0.090306  ] 誤差值: 4.974239251387037
權重: [-0.65599455  1.87075946] 誤差值: 42.506977108508146
權重: [ 0.21866357 -0.0474736 ] 誤差值: 5.205609191362866
權重: [0.15143398 0.090306  ] 誤差值: 4.974239042096608
權重: [0.15143398 0.090306  ] 誤差值: 4.9742391991924615
權重: [-0.02299105  0.6358384 ] 誤差值: 7.654585074135182
權重: [0.18606189 0.12208013] 誤差值: 3.19586441838291
權重: [0.27253659 0.14873505] 誤差值: 1.3813769608999134
權重: [0.38502341 0.14107634] 誤差值: 0.3635538073552802
權重: [0.51417639 0.09750341] 誤差值: 0.36764362006590945
權重: [0.77281568 0.02047292] 誤差值: 0.4145490957243804
權重: [ 1.29069006 -0.11291002] 誤差值: 0.18690958394024526
權重: [0.75305968 0.04373553] 誤差值: 0.36778566210566754
權重: [0.5089764  0.32024402

In [13]:
def ttttest(w):
    v = 0
    for i in range(len(w)):
        v = v + ( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
        - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) )**2
    return v

In [85]:

for i in range(1,11):
    a = i/10
    print(a,ttttest([a,1-a])) 

0.1 6.0665077129419736
0.2 3.843225607347518
0.3 2.3303515734980462
0.4 1.2695095364017597
0.5 0.5675298667580613
0.6 0.1756348971202411
0.7 0.020188468598254558
0.8 0.0048025470511581896
0.9 0.048384805317532455
1.0 0.10370583165756968
